# CSC3831 Final Assessment - Part I: Data Engineering



In [1]:
# Loading in standard packages for analysis, feel free to add an extra packages you'd like to use here
import random
import pandas as pd
import numpy as np
import seaborn as sns
import missingno as msno
# Loading in the corrupted dataset to be used in analysis and imputation
houses_corrupted = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/IMPUTATION/TARGET-DATASETS/CORRUPTED/HOUSES/houses_0.1_MAR.csv', header=0)
# Remove an artifact from the dataset
houses_corrupted.drop(["Unnamed: 0"], axis=1, inplace=True)

Above we've loaded in a corrupted version of a housing dataset. The anomalies need to be dealt with and missing values imputed.

### 1. Data Understanding [7]
- Perform ad hoc EDA to understand and describe what you see in the raw dataset
  - Include graphs, statistics, and written descriptions as appropriate
  - Any extra information about the data you can provide here is useful, think about performing an analysis (ED**A**), what would you find interesting or useful?
- Identify features with missing records, outlier records


##Taking a First Look at the Data
I am beginning my exploratory data analysis by examining the basic information of the `houses_corrupted` dataset using the `info()` function. This provides an overview of the dataset, including the total number of entries, the number of columns, the data types for each column, and the count of non-null values per column. This initial inspection is essential as it reveals any potential missing values and helps determine if further data cleaning is needed.

In [ ]:
houses_corrupted.info()

From the intial inspection using `info()`, I observed that there are missing values in columns like `median_income`, `housing_median_age`, and population, as these columns have 18576 non-null counts compared to the total of 20640 entries. To further explore the structure and contents of the dataset, I used the `head(10)` function to view the first 10 rows.

In [ ]:
houses_corrupted.head(10)

From examining the first 10 rows of the dataset, it is evident there are missing values, particularly in columns like `population` (as seen in rows 2 and 3 where the values are shown as `NaN`). This initial observation has suggested that data imputation may be required to address missing values for further analysis.

## Visualizing Relationships, Distributions and Statistical Summaries

Since the `houses_corrupted` dataset consists entirely of numerical values, I generated a pair plot using the `sns.pairplot` function to examine the center, spread, and skew of data. The pair plot enables us to visually explore the distributions and pairwise relationships between attributes, helping to identify trends, clusters, or unusual patterns that could impact the data analysis.

In [ ]:
sns.pairplot(houses_corrupted)

Due to large volume of data points, it is challenging to determine the skew for each attribute directly from the pair plot alone. The dense overlapping points make it difficult to discern finer details in data distribution. To address this, I created a density plot for each attribute using the `sns.kdeplot` function, which provides a clearer view of the skewness of the data and distribution shape.

In [ ]:
sns.kdeplot(houses_corrupted['median_house_value'])

In [ ]:
sns.kdeplot(houses_corrupted['median_income'])

In [ ]:
sns.kdeplot(houses_corrupted['housing_median_age'])

In [ ]:
sns.kdeplot(houses_corrupted['total_rooms'])

In [ ]:
sns.kdeplot(houses_corrupted['total_bedrooms'])

In [ ]:
sns.kdeplot(houses_corrupted['population'])

In [ ]:
sns.kdeplot(houses_corrupted['households'])

In [ ]:
sns.kdeplot(houses_corrupted['latitude'])

In [ ]:
sns.kdeplot(houses_corrupted['longitude'])

The density plots show that `median_house_value`, `median_income`, `total_rooms`, `total_bedrooms`, `population`, and `households` are all right-skewed with the majority of values clustered at lower ranges and a long tail extending to higher values. This right-skewness suggests a high concentration of data points at the lower end of each variable's range, with fewer outliers on the higher end.

Furthermore, the density plot for `housing_median_age` shows multiple peaks, indicating a multimodal distribution rather than a simple skew. Unlike other variables, `housing_median_age` does not display a strong right or left skew but instead has several prominent peaks, showing varying concentrations of housing ages across different ranges.

Additionally, the `latitude` and `longitude` plots display bimodal distributions, each with two distinct peaks. This indicates that there are two main clusters of data points in these attributes.

Next, I used the `describe` method to view the summary statistics of the numeric values.

In [2]:
houses_corrupted.describe()

,median_house_value,median_income,housing_median_age,total_rooms,total_bedrooms,population,households,latitude,longitude
count,20640.000000,18576.000000,18576.000000,20640.000000,20640.000000,18576.000000,20640.000000,20640.000000,20640.000000
mean,206855.816909,3.929958,28.324182,2635.763081,537.898014,1488.069283,499.539680,35.631861,-119.569704
std,115395.615874,1.964296,12.584914,2181.615252,421.247906,1170.585810,382.329753,2.135952,2.003532
min,14999.000000,0.499900,1.000000,2.000000,1.000000,3.000000,1.000000,32.540000,-124.350000
25%,119600.000000,2.560300,18.000000,1447.750000,295.000000,839.000000,280.000000,33.930000,-121.800000
50%,179700.000000,3.572400,28.000000,2127.000000,435.000000,1227.000000,409.000000,34.260000,-118.490000
75%,264725.000000,4.870050,37.000000,3148.000000,647.000000,1803.000000,605.000000,37.710000,-118.010000
max,500001.000000,15.000100,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,41.950000,-114.310000


In [ ]:
# Assuming `houses_corrupted` is the DataFrame you want to calculate MAD for

houses_corrupted.columns[:-1]

# Initialize an empty DataFrame with the column names (only numerical columns)
houses_MAD = pd.DataFrame(columns=houses_corrupted.columns[:-1])

# List to store MAD values for each attribute
mads = []

# Calculate MAD for each numerical attribute in houses_corrupted
for attribute in houses_corrupted.columns[:-1]:
    mad = 1.483 * abs(houses_corrupted[attribute] - houses_corrupted[attribute].median()).median()
    mads.append(mad)

# Store the MAD values in the DataFrame
houses_MAD.loc[0] = mads

# Print the MAD values
print(houses_MAD)


In [3]:
houses_corrupted.corr()

,median_house_value,median_income,housing_median_age,total_rooms,total_bedrooms,population,households,latitude,longitude
median_house_value,1.000000,0.694887,0.097929,0.134153,0.050594,-0.027855,0.065843,-0.144160,-0.045967
median_income,0.694887,1.000000,-0.120147,0.198818,-0.009499,0.006298,0.012754,-0.096861,-0.008902
housing_median_age,0.097929,-0.120147,1.000000,-0.372323,-0.329757,-0.305052,-0.312948,0.011372,-0.106438
total_rooms,0.134153,0.198818,-0.372323,1.000000,0.929893,0.857515,0.918484,-0.036100,0.044568
total_bedrooms,0.050594,-0.009499,-0.329757,0.929893,1.000000,0.877178,0.979829,-0.066318,0.068378
population,-0.027855,0.006298,-0.305052,0.857515,0.877178,1.000000,0.907096,-0.107525,0.099797
households,0.065843,0.012754,-0.312948,0.918484,0.979829,0.907096,1.000000,-0.071035,0.055310
latitude,-0.144160,-0.096861,0.011372,-0.036100,-0.066318,-0.107525,-0.071035,1.000000,-0.924664
longitude,-0.045967,-0.008902,-0.106438,0.044568,0.068378,0.099797,0.055310,-0.924664,1.000000


In [4]:
# Missing values per column
missing_data = houses_corrupted.isnull().sum()
missing_percentage = (missing_data / houses_corrupted.shape[0]) * 100
missing_info = pd.DataFrame({'Missing Values': missing_data, 'Percentage': missing_percentage})
print(missing_info)

                    Missing Values  Percentage
median_house_value               0         0.0
median_income                 2064        10.0
housing_median_age            2064        10.0
total_rooms                      0         0.0
total_bedrooms                   0         0.0
population                    2064        10.0
households                       0         0.0
latitude                         0         0.0
longitude                        0         0.0


In [ ]:
# Outlier Detection using IQR with accurate counting
outliers = pd.DataFrame()
outlier_counts = {}

for column in houses_corrupted.select_dtypes(include=[np.number]).columns:
    Q1 = houses_corrupted[column].quantile(0.25)
    Q3 = houses_corrupted[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Remove NaNs before calculating outliers
    column_data = houses_corrupted[column].dropna()
    outliers[column] = column_data[(column_data < lower_bound) | (column_data > upper_bound)]

    # Count the outliers for each column
    outlier_counts[column] = outliers[column].notna().sum()

# Print outlier counts for each column
print("Outliers Count per Column:")
for column, count in outlier_counts.items():
    print(f"{column}: {count} outliers")


In [ ]:
sns.boxplot(x=houses_corrupted['population'])

### 2. Outlier Identification [10]
- Utilise a statistical outlier detection approach (i.e., **no** KNN, LOF, 1Class SVM)
- Utilise an algorithmic outlier detection method of your choice
- Compare results and decide what to do with identified outleirs
  - Include graphs, statistics, and written descriptions as appropriate
- Explain what you are doing, and why your analysis is appropriate
- Comment on benefits/detriments of statistical and algorithmic outlier detection approaches


### 3. Imputation [10]
- Identify which features should be imputed and which should be removed
  - Provide a written rationale for this decision
- Impute the missing records using KNN imputation
- Impute the missing records using MICE imputation
- Compare both imputed datasets feature distributions against each other and the non-imputed data
- Build a regressor on all thre datasets
  - Use regression models to predict house median price
  - Compare regressors of non-imputed data against imputed datas
  - **Note**: If you're struggling to compare against the original dataset focus on comparing the two imputed datasets against each other


In [ ]:
# Use this dataset for comparison against the imputed datasets
houses = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/IMPUTATION/TARGET-DATASETS/ORIGINAL/houses.csv', header=0)

### 4. Conclusions & Throughts [3]
- Disucss methods used for anomaly detection, pros/cons of each method
- Disucss challenges/difficulties in anomaly detection implementation
- Discuss methods used for imputation, pros/cons of each method
- Discuss challenges/difficulties in imputation implementation